# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove the `raise NotImplementedError()` statement.

The first two questions are on phase estimation and the rest is about order finding algorithm.

## Question 1 (2 points)

Complete the function `create_operator` that returns a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$.

The function `create_operator` has

- Input: $\phi$
- Returns: Controlled $U$ gate with the properties described above

Don't create any circuit. Only a gate should be returned.

In [89]:
from cirq import CZPowGate

def create_operator(phi):

    exponent = 2 * phi
    CU = CZPowGate(exponent=exponent)
    # YOUR CODE HERE

    # Do not modify anything below this line
    return CU

In [90]:
# You can use this code to test your function by checking the bottom right corner of the unitary matrix
import cirq
from cmath import exp, pi

def test(phi):
    CU = create_operator(phi)
    unitary_matrix = cirq.unitary(CU)
    print(unitary_matrix)

phi=3/5
test(phi)
print(exp(2*pi*phi*1j))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.80901699-0.58778525j]]
(-0.8090169943749476-0.587785252292473j)


In [91]:
# hidden tests in this cell will be used for grading.

In [92]:
# hidden tests in this cell will be used for grading.

## Question 2 (8 points)

Complete the function `estimate_phi` such that given a controlled $U$ gate where $U$ is an operator with eigenvector $|1\rangle$ and eigenvalue $e^{2\pi i \phi}$, it estimates and returns $\phi$.

The function `estimate_phi` has

- Input: Controlled $U$ gate
- Returns: Estimate for phi

You are given iqft and qpe algorithms and you can use them in your solution. Let the size of the first register equal 10.

In [93]:
def iqft(n,qubits,circuit):

    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)

    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step

        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):

    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))

    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))

    #Apply inverse QFT
    iqft(t,control,circuit)

In [94]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def estimate_phi(mystery):


    #Create cirucit
    circuit = cirq.Circuit()

    # START CODE 
    
    # 1. Set the size of the counting (first) register as specified
    t = 10

    # 2. Define the qubits
    control_qubits = cirq.LineQubit.range(t)
    target_qubit = cirq.LineQubit(t)  # A single target qubit, indexed after control

    # 3. Prepare the eigenvector |1>
    # The problem states U has eigenvector |1>. QPE requires
    # the target register to be in this state.
    circuit.append(cirq.X(target_qubit))

    # 4. Call the provided QPE algorithm function
    # The 'mystery' gate is the Controlled-U (CU)
    # The 'target' argument in qpe() expects a list of qubits
    qpe(t, control_qubits, [target_qubit], circuit, mystery)

    # 5. Measure the control register to read out the phase
    circuit.append(cirq.measure(*control_qubits, key='result'))
    
    # --- (End of YOUR CODE HERE) ---
    # END CODE
    # YOUR CODE HERE
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    #Most frequent observation
    freq = list(samples.histogram(key='result').keys())[0]

    return freq/2**t

In [95]:
import math
# You can use this code to test your function by different operators
def test_qpe(phi):
    operator = CZPowGate(exponent=2*phi)
    return estimate_phi(operator)

assert(math.isclose(test_qpe(0.23),0.23,rel_tol=1e-2))

In [96]:
# hidden tests will be used for grading.

## Question 3 (1 point)

Complete the function `calcL` such that given $N$, it returns the size $L$ of the target register. Note that $L = \log_2 N$.

The function `calcL` has

- Input: $N$
- Returns: size of the target register

In [97]:
import math
def calcL(N):

    L = math.ceil(math.log2(N))
    # YOUR CODE HERE
    return L

In [98]:
# You can use this cell to test your function

print(calcL(30))
print(calcL(62))

5
6


In [99]:
# hidden tests will be used for grading.

## Question 4 (1 point)

Complete the function `calct` such that given $N$, it returns the size $t$ of the control register. Note that $t = 2L + 1 + \big \lceil \log \big( 2 + \frac{1}{2\epsilon} \big) \big \rceil$. Let $\epsilon=0.1$.

The function `calct` has

- Input: $N$
- Returns: size of the control register

In [100]:
import math
def calct(N):

    L=calcL(N)
    t=2*L+1+math.ceil(math.log2(2+(1/(2*0.1))))
    # YOUR CODE HERE
    return t

In [101]:
# You can use this cell to test your function

print(calct(30)) #For 30 the answer is 14
print(calct(62)) #For 62 the answer is 16

14
16


In [102]:
# hidden tests will be used for grading.

## Question 5 ( 2 points)

Complete the function `create_regs` such that given the size of the control and target registers, it returns the control and target registers.

The function `create_regs` has

- Input: $L$ and $t$
- Returns: Two registers `control` and `target`

In [103]:
import cirq
def create_regs(t,L):

    control = cirq.LineQubit.range(t)
    target = cirq.LineQubit.range(L)
    # YOUR CODE HERE
    return control, target

In [104]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect

In [105]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect

## Question 6 (8 points)

Complete the function `order` so that given $x$ and $N$, it returns the histogram of samples from the order finding circuit.

The function `order` has

- Input: $x$ and $N$
- Returns: A histogram of samples

Notes:

- You are given a function named $U_x$ which implements $U_x |y\rangle \rightarrow |xy {\mod N}\rangle $ and returns its controlled version. Run the following cell to load the function. In order to use the function you should pass $x$ and $N$ as parameter.

    <pre>CU=Ux(x,N)</pre>

- You are also given the code for `qpe` and `iqft`.

In [106]:
import numpy as np
def Ux(x,N):

    k=1
    while(N>2**k):
        k=k+1

    u = np.zeros([2**k, 2**k], dtype = int)

    for i in range(N):
        u[x*i%N][i]=1
    for i in range(N,2**k):
        u[i][i]=1


    XU = cirq.MatrixGate(u).controlled()
    return XU

In [107]:
import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate
def iqft(n,qubits,circuit):

    #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)

    #For each qubit
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1 #Decrement at each step

        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]),strategy = InsertStrategy.NEW)

def qpe(t,control, target, circuit, CU):

    #Apply Hadamard to control qubits
    circuit.append(cirq.H.on_each(control))

    #Apply CU gates
    for i in range(t):
        #Obtain the power of CU gate
        CUi = CU**(2**i)
        #Apply CUi gate where t-i-1 is the control
        circuit.append(CUi(control[t-i-1],*target))

    #Apply inverse QFT
    iqft(t,control,circuit)

In [108]:
import cirq
from cirq import X, SWAP

def order(x,N):
    #Create a circuit
    circuit = cirq.Circuit()
    
    # START CODE 
    
    # 1. Get the controlled U_x gate
    CU = Ux(x, N)

    # 2. Determine the number of qubits for the target register (k)
    # The CU gate from Ux() has 1 control and 'k' target qubits.
    # So, k = (total qubits in CU) - 1.
    k = CU.num_qubits() - 1

    # 3. Determine the number of qubits for the control register (t)
    # For order finding, we need high precision, so we use
    # t = 2k (which gives 2*log(N) bits of precision).
    t = 2 * k

    # 4. Define the two qubit registers
    # The measurement code below uses a variable named 'control'
    control = cirq.LineQubit.range(t)
    target = cirq.LineQubit.range(t, t + k)

    # 5. Prepare the target register in the state |1>
    # The |1> state is a superposition of all eigenvectors of U_x.
    # We create |1> (which is |00...01>) by applying an X gate
    # to the last qubit of the target register.
    circuit.append(cirq.X(target[-1]))

    # 6. Call the provided QPE function to build the circuit
    # qpe(t, control, target, circuit, CU)
    qpe(t, control, target, circuit, CU)
    
    # --- (End of YOUR CODE HERE) ---
    # END CODE
    # YOUR CODE HERE

    #Measure the control register
    circuit.append(cirq.measure(*control, key='result'))

    #Sample the circuit
    s=cirq.Simulator()
    samples=s.run(circuit, repetitions=1000)

    # Return a histogram of samples
    results= samples.histogram(key='result')
    return results

In [109]:
#You can use this cell to test your function

results = order(4,31)
print(results)

Counter({0: 227, 205: 161, 819: 158, 614: 114, 410: 108, 615: 56, 409: 45, 204: 14, 613: 13, 411: 12, 820: 9, 616: 9, 206: 6, 408: 6, 612: 5, 818: 5, 611: 4, 203: 4, 618: 3, 400: 2, 202: 2, 194: 2, 826: 2, 406: 2, 413: 2, 407: 2, 617: 2, 829: 1, 474: 1, 418: 1, 414: 1, 351: 1, 808: 1, 213: 1, 605: 1, 813: 1, 621: 1, 821: 1, 547: 1, 569: 1, 609: 1, 817: 1, 864: 1, 215: 1, 423: 1, 403: 1, 207: 1, 302: 1, 419: 1, 513: 1, 571: 1, 393: 1})


In [110]:
# hidden tests will be used for grading.
# If this cell results in an error, your implementation is incorrect